In [5]:
import numpy as np
import pandas as pd
import gc
import random
import string
import os

import torch
import gc
gc.collect()
torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split

import fasttext
import fasttext.util

from transformers import BertForSequenceClassification, AdamW,BertTokenizer,get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

### Veri Okuma

In [2]:
df_dataset = pd.read_csv("../dataset/dataset_without_stopwords.csv").drop(columns=["baslik"])

In [6]:
temp_lyrics = [lyrics.replace("\n", "[SEP]").rstrip("[SEP]") for lyrics in df_dataset.loc[:,"sarki_sozu"]]
df_dataset["sarki_sozu"] = temp_lyrics

##### Load Fasttext Embeddings

In [6]:
path = '../models/cc.tr.300.bin'
if (not os.path.exists(path)):
    fasttext.util.download_model('tr', if_exists='ignore')
    os.replace("cc.tr.300.bin", path)
    os.remove("cc.tr.300.bin.gz")

ft = fasttext.load_model(path)


 (100.00%) [==================================================>]                                                 ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]> 

In [7]:
ft.get_dimension()

300

In [25]:
split_chars = ["[CLS]", " ", "[SEP]"]

splitted_words_for_all_lyrics_in_data = []
for lyrics in df_dataset.loc[:, "sarki_sozu"]:
    words = []
    for char in split_chars:
        lyrics = lyrics.replace(char, " ")
    words = lyrics.split()
    splitted_words_for_all_lyrics_in_data.append(words)
kelimeler = splitted_words_for_all_lyrics_in_data

In [27]:
sent_list=[]
for lyrics in kelimeler:
    temp_sent_list = []
    for word in lyrics:
        wv=ft.get_word_vector(word)
        temp_sent_list.append(wv)
    sent_list.append(temp_sent_list)

import numpy as np 
sent_embed= np.mean(sent_list[0],axis=0)